In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore")
# plt.style.use('dark_background')

In [ ]:
from utils import datasets
from utils.active_forests import ALIF, BALEIF, BALIF, RandomForest
from utils.utils import *

# simulations

In [ ]:
sim_params = {"n_runs": 24, "max_iterations": 50, "size_query": None, "jobs":8}

In [ ]:
def run_simulations(model_name, ensamble_pred=None, query_strat=None):
    model_class = {
        "BALIF": BALIF,
        "BALEIF": BALEIF,
        "ALIF": ALIF,
        "RF": RandomForest,
    }.get(model_name)

    if ensamble_pred is None:
        ensamble_pred = {
            "BALIF": "naive",
            "BALEIF": "naive",
            "ALIF": "piecewise",
            "RF": "naive",
        }.get(model_name)

    if query_strat is None:
        query_strat = {
            "BALIF": "margin",
            "BALEIF": "margin",
            "ALIF": "anomalous",
            "RF": "random",
        }.get(model_name)

    for i, data_name in enumerate(pbar := tqdm(datasets.datasets_names_short)):
    #for i, data_name in enumerate(pbar := tqdm(["cover"])):
        pbar.set_description(f"dataset {data_name}")
        data, labels = datasets.load_dataset(data_name)

        model = model_class(query_strategy=query_strat, ensamble_prediction=ensamble_pred)
        sim_logs = evaluate_performance(model, data, labels, **sim_params)

        path = f"performance/{data_name}_{model_name}_query-{query_strat}_pred-{ensamble_pred}"
        np.save(path, sim_logs, allow_pickle=True)

In [ ]:
model_name_list = [
    "BALEIF",
    "BALIF",
    "ALIF",
    #"RF",
]
for model_name in (pbar := tqdm(model_name_list)):
    pbar.set_description(f"model {model_name}")
    run_simulations(model_name)